   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 4.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import set_seed
import torch
from typing import Callable
import random
import os
from guidance import models, gen, select, system, assistant, user

/w5home/lxu3/anaconda3/envs/kmGPT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configuration variables for this whole notebook
class config:
    seed = 42
    model = "Open-Orca/Mistral-7B-OpenOrca"
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
    top_k = 20
    top_p = 0.95
    do_sample = True
    num_return_sequences = 1
    max_new_tokens = 500
    temperature = 0.8
    repetition_penalty = 1.2
    penalty_alpha=0.6


In [4]:
import os
os.environ['HF_HOME'] = '/w5home/lxu3/kmGPT'

In [1]:
import huggingface_hub

In [26]:
mistral = models.TransformersChat("Mistral-7B-OpenOrca", torch_dtype = torch.bfloat16, trust_remote_code = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.20s/it]


In [14]:
model = mistral.model_obj

In [15]:
tokenizer = mistral._orig_tokenizer

In [17]:
model.save_pretrained("./mistral-open-orca")

In [20]:
tokenizer

LlamaTokenizer(name_or_path='Open-Orca/Mistral-7B-OpenOrca', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
}

In [24]:
model = AutoModelForCausalLM.from_pretrained("Mistral-7B-OpenOrca")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.58s/it]


In [25]:
tokenizer.from_pretrained("Mistral-7B-OpenOrca")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaTokenizer(name_or_path='Mistral-7B-OpenOrca', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
}

In [6]:
data = pd.read_csv("./src/data.csv")
data

,Unnamed: 0,pmid,abstract,label
0,0,23956253,PMID: 23956253 Text: The first aim was to crit...,0
1,1,23444397,PMID: 23444397 Text: Niacin has potentially fa...,0
2,2,28886926,"PMID: 28886926 Text: In 2016, the American Col...",0
3,3,27701660,PMID: 27701660 Text: Low-density lipoprotein c...,1
4,4,19095139,PMID: 19095139 Text: This secondary analysis f...,0
5,5,21095263,PMID: 21095263 Text: Lowering low-density lipo...,0


In [7]:
def retrieveZeroShotCoTPrompt(hyp: str, abstract: str) -> str:
  zero_shot_prompt = f"""
    Hypothesis: {hyp}
    Abstract: {abstract}

    Determine whether or not this abstract is relevant for scientifically evaluating the provided hypothesis.
    A relevant abstract should either support the given hypothesis or have evidence to refute the hypothesis.
    A relevant abstract must directly comment on the hypothesis.

    Let us think through this step by step.
  """
  return zero_shot_prompt

In [8]:
hyp = "ezetimibe may effectively alleviate or target key pathogenic mechanisms of diabetes potentially offering therapeutic benefits or slowing disease progression."
sys_prompt = "You are an incredibly brilliant biomedical researcher who has spent their lifetime reading all the papers in PubMed. You are focused on assisting other researchers in evaluating suggested hypotheses given abstracts in PubMed."

In [9]:
prompt = retrieveZeroShotCoTPrompt(hyp, data["abstract"][2])

In [10]:
with system():
    lm = mistral + sys_prompt

with user():
    lm += prompt

with assistant():
    lm += gen(max_tokens = 500, temperature = config.temperature, name = "chain_of_thought")

with user():
    lm += "Give a score of either 0: (Not relevant) or 1: (Relevant) for the above abstract. Answer: " + select([0, 1], name = "answer")

KeyboardInterrupt: 